# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298273839144                   -0.85    5.2
  2   -8.300190458534       -2.72       -1.26    1.2
  3   -8.300448354252       -3.59       -1.89    3.2
  4   -8.300462019330       -4.86       -2.75    1.4
  5   -8.300464215085       -5.66       -3.07    6.1
  6   -8.300464415677       -6.70       -3.25    1.2
  7   -8.300464525356       -6.96       -3.38    5.0
  8   -8.300464582671       -7.24       -3.50    1.0
  9   -8.300464627674       -7.35       -3.70    1.0
 10   -8.300464639636       -7.92       -3.90    1.0
 11   -8.300464643666       -8.39       -4.29    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64200908138                   -0.70    5.4
  2   -16.67861267492       -1.44       -1.14    1.4
  3   -16.67923303031       -3.21       -1.88    3.8
  4   -16.67928012407       -4.33       -2.77    3.2
  5   -16.67928595819       -5.23       -3.14    5.2
  6   -16.67928621392       -6.59       -3.54    1.9
  7   -16.67928622209       -8.09       -4.07    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32367509373                   -0.56    7.0
  2   -33.33227847017       -2.07       -1.00    1.0
  3   -33.33415119759       -2.73       -1.75    4.5
  4   -33.33598983347       -2.74       -2.33    2.2
  5   -33.33691580053       -3.03       -2.56   17.0
  6   -33.33693802481       -4.65       -2.92    2.8
  7   -33.33694388700       -5.23       -3.76    2.6
  8   -33.33694392440       -7.43       -4.11    5.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298557492965                   -0.85    5.5
  2   -8.300266091800       -2.77       -1.59    1.0
  3   -8.300461803752       -3.71       -2.69    4.5
  4   -8.300453606161   +   -5.09       -2.74    9.2
  5   -8.300464051566       -4.98       -3.38    2.6
  6   -8.300464589661       -6.27       -3.94    1.8
  7   -8.300464643293       -7.27       -4.81    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32681100500                   -0.56    6.9
  2   -33.31272363179   +   -1.85       -1.27    1.2
  3   -11.71617334980   +    1.33       -0.40    8.4
  4   -33.33211793574        1.33       -1.87    6.8
  5   -33.25463820862   +   -1.11       -1.33    4.4
  6   -33.33510317653       -1.09       -2.14    5.4
  7   -33.33388506789   +   -2.91       -2.30    2.1
  8   -33.29994570135   +   -1.47       -1.78    5.0
  9   -33.33566257758       -1.45       -2.31    4.1
 10   -33.33676552453       -2.96       -2.86    2.1
 11   -33.33691806740       -3.82       -3.22    3.4
 12   -33.33693260171       -4.84       -3.47    3.4
 13   -33.33694352957       -4.96       -4.08    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.